#### How to use the Code

1. install whisper: pip install git+https://github.com/openai/whisper.git
2. install ffmpeg

    * Mac: brew install ffmpeg
    * Windows: choco install ffmpeg (using the powershell as an administrator after successfully having installed chocolatey)
    
        * folgenden befehl in windows Powershell ausführen:
        
            Set-ExecutionPolicy Bypass -Scope Process -Force; `

            [System.Net.ServicePointManager]::SecurityProtocol = `
        
            [System.Net.ServicePointManager]::SecurityProtocol -bor 3072; `
        
            iex ((New-Object System.Net.WebClient).DownloadString('https://community.chocolatey.org/install.ps1'))
        
        * dann folgenden Befehl ausführen: 

            choco install ffmpeg




In [ ]:
%pip install transformers torchaudio


#### Using Whisper Base (200 MB)

In [10]:
from transformers import pipeline
import pprint as pp

pipe = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-base"
)

result = pipe("memo.wav", return_timestamps=True)
pp.pprint(result)



Device set to use cuda:0
c:\Users\Anwender\anaconda3\envs\Graph_RAG_BFH\Lib\site-packages\transformers\models\whisper\generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


{'chunks': [{'text': ' Die ist komm nach ständiger Rechtsprechung nur dann in '
                     'Betracht, wenn eine eindeutige',
             'timestamp': (0.0, 8.96)},
            {'text': ' Unbilligkeit von erheblichen Gewicht vorliegt.',
             'timestamp': (8.96, 11.24)},
            {'text': ' Eine solche liegt vor, wenn aufgrund des derartige '
                     'Umstände des Einzelfalls, die sie aus dem',
             'timestamp': (11.24, 16.16)},
            {'text': ' groben Maßstab des Paragraphen 29 Absatz 1 Nummer 1 '
                     'Gewerbestolgesetz allgemein ergebende',
             'timestamp': (16.16, 21.6)},
            {'text': ' Unbilligkeit offensichtlich übertroffen wird.',
             'timestamp': (21.6, 23.8)},
            {'text': '', 'timestamp': (23.8, 0.0)},
            {'text': ' Zum Beispiel BFH-O-Tai 52511 2009, große E, große R18 '
                     'aus 08. Dies ist zum Beispiel dann der Fall, wenn der '
                     'St

In [11]:
#put the utput text in a dataframe in the left of two columns
import pandas as pd

# create two columns one named text first the second named text second
df = pd.DataFrame({
    "text first": [result['text']],
    "text second": [""]
})

df

,text first,text second
0,Die ist komm nach ständiger Rechtsprechung nu...,


#### Using Whisper Large (6.17 GB)

* 1:30 min file wird innerhalb von 11 sec transkribiert 
* 15:00 min file wird innerhalb von 1:10 min transkribiert

In [13]:
from transformers import pipeline
import pprint as pp

pipe = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-large"
)

result = pipe("memo.wav", return_timestamps=True)
pp.pprint(result["text"])

del pipe

# GPU-Speicher freigeben
import torch
torch.cuda.empty_cache()

Device set to use cuda:0


(' Dies kommt nach ständiger Rechtsprechung nur dann in Betracht, wenn eine '
 'eindeutige Unbilligkeit von erheblichem Gewicht vorliegt. Eine solche liegt '
 'vor, wenn aufgrund der atypischen Umstände des Einzelfalls, die sich aus dem '
 'groben Maßstab des 29 Abs. 1 Nr. 1 Gewerbesteuergesetz allgemein ergebende '
 'Unbilligkeit offensichtlich übertroffen zum Beispiel BFH-Urteil vom '
 '25.11.2009, Groß I, Groß R, 18, 08. Dies ist zum Beispiel dann der Fall, '
 'wenn der Steuerpflichtige in der Betriebsstätte, die nach Art und Umfang von '
 'erheblicher Bedeutung ist, auf Dauer und ausschließlich Leiharbeitnehmer '
 'einsetzt, anstatt, wie sonst im Allgemeinen üblich, die eingesetzten '
 'Arbeitskräfte selbst zu beschäftigen und dadurch der Gemeinde, in der diese '
 'Betriebsstätte unterhalten wird, das Gewerbesteueraufkommen aus dieser '
 'Betriebsstätte vollständig entgeht. Eine Unbilligkeit im Sinne des 33 Absatz '
 '1 Satz 1 Gewerbesteuergesetz kommt auch dann in Betracht, wenn d

In [15]:
# now put the text in the dataframe named df into the first row of the colun text second
df.at[0, 'text second'] = result['text']

df

import openpyxl
#export the dataframe in excel format
#df.to_excel("output.xlsx", index=False)

#### Using Whisper-lage-v3-turbo (1.7 GB)

* 1:30 min file in 3 sec
* 15:00 file in 16 sec

Problem: häufige Halluzination, wenn am Ende der Aufnahme kurz Stille ist.

Dann wird am Ende des Transkripts "Untertitelung des ZDF für funk, 2017" angefügt (siehe dazu im internet)

In [2]:
from faster_whisper import WhisperModel
import torch, gc, pprint as pp

def transcribe_full(path):
    model = WhisperModel(
        "large-v3-turbo",
        device="cuda",
        compute_type="float16",
        cpu_threads=4,
        num_workers=1,
    )

    segments, info = model.transcribe(path, word_timestamps=False, beam_size=5)

    # Volltext zusammensetzen
    full_text = " ".join(segment.text.strip() for segment in segments)

    # Ressourcen bereinigen
    del model, segments, info
    gc.collect()
    torch.cuda.empty_cache()

    return full_text

text = transcribe_full("memo_lang.wav")
pp.pprint(text)


('Hallo, mein Name ist Hans Müller. Ich berichte über eine Patientin mit dem '
 'anonymisierten Code von Z290361. Die Patientin ist verheiratet, hat einen '
 'Sohn, minus 22 Jahre, Beruf Architekt und eine Tochter, plus 24 Jahre, Beruf '
 'Studentin. Ihr Mann ist berentet. Sie arbeitet aktuell 40 Stunden als '
 'Betreuungsassistentin in Teilzeit. Gemeinsam mit ihrem Sohn und ihrer '
 'Tochter und ihrem Ehemann bewohnt sie ein Einfamilienhaus. Finanzielle '
 'Sorgen bestehen nicht. Zur Symptomatik, die Patientin berichtet seit '
 'mehreren Jahren unter Schlafstörungen, innerer Unruhe, gereizter Stimmung, '
 'Versagensgefühlen, Schuldgefühlen, verminderten Selbstwort, emotionaler '
 'Instabilität, Energieverlust, Konzentrationsstörungen, Verlust der '
 'Lebensfreude und Zukunftsängsten zu leiden. Sie selbst gibt an, in einem '
 'Pflegeheim zu arbeiten und täglich die Arbeit kranker Kollegen oder der '
 'Arbeit von unbesetzten Stellen mitarbeiten zu müssen. Ihr Mann leidet seit '
 'vielen

In [17]:
#take the df and add a new column named text third and put the text in thr first row of the column

df['text third'] = ""
df.at[0, 'text third'] = text
# export the dataframe in excel format
df.to_excel("output_full.xlsx", index=False) 